In [1]:
!python3 --version

Python 3.10.12


#### fix for could not import -lmza
https://github.com/lucidrains/imagen-pytorch/issues/92 

#### Fix for Can't import datasets AttributeError: partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

Restart juypyter and kernel


In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
import helper
import time
helper.check_gpu_capacity()

GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB


In [4]:
from datasets import load_dataset, load_from_disk
from random import randrange



In [5]:

# Load dataset from the hub
dataset = load_from_disk("generated-dataset/llama-7-int4-dolly-filtered")

dataset


Dataset({
    features: ['instruction', 'response'],
    num_rows: 687
})

In [6]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
Given this passage about formula, what is the ratio of the ingredients in the inoculum sub recipe?
--> response
Characteristic for partially fermented formulas is the presence of a second BP as can be seen in the screenshot below.  This second BP is the fermented BP, also referred to as Lactofidus. 

If there is already an existing fermented BP, this existing BP can be inserted. Otherwise, a second BP template can be inserted. 

Once everything with personal code was copied to the PD workbench, one will see an image similar to the screen shown above. Thus, in the final dry blending recipe of partially fermented formula’s, there will be a fermented BP, an unfermented BP, a PX and lactose (Masterbatch) and/or other ingredients which will all be dry blended.

	4.2 Formulating Fermented BP in Bestmix

In the next step, RM ingredients should be added to each of the templates considering the general procedure for recipe formulation. As the fermented BP should be formulated ac

In [7]:
# list(filter(lambda x :x != '', list(map(lambda x:((x['category'], len(x['context'])) if len(x['context']) > 5000 else ''), dataset))))

Based on our use case, we want to focus on 'general_qa' and  'information_extraction tasks.,

In [8]:
dataset = dataset.filter(lambda x: x['instruction'] != '')

# print(set(dataset['instruction']))
dataset

Loading cached processed dataset at /home/ubuntu/ai-project/generated-dataset/llama-7-int4-dolly-filtered/cache-e62e99f4975fd97d.arrow


Dataset({
    features: ['instruction', 'response'],
    num_rows: 687
})

In [9]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
Please provide me a summary of the product description of LeopARd.
--> response
LeopARd  product  is  a  new  combination  of  the  food  additive  LBG  (0.4g/100ml)  as  formula  thickener  and 
scGOS/lcFOS (0.4g/100ml (ratio 9:1)) in 25.9% fermented infant formula. The composition of the product is 
in  line  with  the  EC  directive  for  FSMP  (1999/21/EC).  This  product  is  a  partly  fermented,  thickened  infant 
formula  supplemented  with  prebiotics  for  the  dietary  management  of  regurgitation  in  infants  aged  0-12 
months. For the dietary management of regurgitation, an anti-regurgitation (AR) product (Gallia ARTM) was 
available on the market for more than 15 years that contains locust bean gum (LBG, 0.4 g/100 ml) in a 15% 
fermented  formula  but  without  the  non-digestible  carbohydrates  (NDCs)  scGOS/lcFOS.    As  infants  with 
regurgitation frequently display other functional gastrointestinal disorder related signs and symptoms (hard 
stool

Split the dataset into train, eval, and test sets

In [10]:

shuffled_dataset = dataset.shuffle(seed=42)
shuffled_dataset = shuffled_dataset.train_test_split(test_size=0.1)

train_dataset = shuffled_dataset['train']
test_dataset = shuffled_dataset['test']

train_eval_dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_eval_dataset['train']
eval_dataset = train_eval_dataset['test']


# Print the sizes of the train, eval, andtest sets
print("Dataset size:", len(dataset))
print("Train size:", len(train_dataset))
print("Eval size:", len(eval_dataset))
print("Test size:", len(test_dataset))

Loading cached shuffled indices for dataset at /home/ubuntu/ai-project/generated-dataset/llama-7-int4-dolly-filtered/cache-ac25ce4f41b2fe27.arrow


Dataset size: 687
Train size: 494
Eval size: 124
Test size: 69


### Preprocess the Dialog-Summary Dataset
We need to convert the instruction-response (prompt-response) pairs into explicit instructions for the LLM. 

Prepend an instruction to the start of the dialog with Answer this question and to the start of the question with question as follows:

Training prompt (Question):
Answer this question.

    What is the capital of France?

Answer: 
Training response (answer):

    The capital of France is Paris.




Let's test our formatting function on a random example.

### Instruction-tune Llama 2 using trl and the SFTTrainer

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, \
AutoModelForSeq2SeqLM, BitsAndBytesConfig, Trainer

use_flash_attention = False
# COMMENT IN TO USE FLASH ATTENTION
# replace attention with flash attention
# if torch.cuda.get_device_capability()[0] >= 8:
#     from utils.llama_patch import replace_attn_with_flash_attn
#     print("Using flash attention")
#     replace_attn_with_flash_attn()
#     use_flash_attention = True

helper.check_gpu_capacity()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22508 MB
  Used Memory: 7 MB


In [12]:
# Hugging Face model id
model_flan_t5_base_id = "google/flan-t5-base" 

# BitsAndBytesConfig int-4 config
# bnb_config = BitsAndBytesConfig(
#     # load_in_8bit=True,
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )



In [13]:
# Hugging Face model id
model_flan_t5_base_id = "google/flan-t5-base" 

# Load model and tokenizer
model_flan_t5_base = AutoModelForSeq2SeqLM.from_pretrained(model_flan_t5_base_id, 
                                                        # quantization_config=bnb_config, 
                                                        use_cache=False, 
                                                        # device_map="auto",
                                                        torch_dtype=torch.bfloat16
                                                        # device_map={'':torch.cuda.current_device()},
                                                           # device_map=3,
                                                           # device=f'cuda:1'
                                                          )
# model_flan_t5_base.to(f'cuda:1')
tokenizer_flan_t5_base = AutoTokenizer.from_pretrained(model_flan_t5_base_id)
# model_flan_t5_base.hf_device_map

In [14]:
tokenizer_flan_t5_base.model_max_length

512

In [15]:
def format_instruction(sample):
	return f"""### Instruction:
            Answer this question.
            
            ### Question:
            {sample['instruction']}

            ### Answer:
            {sample['response']}
        """


In [16]:
def tokenize_function(sample, tokenizer):
    prompt = f"""### Instruction:
            Answer this question.
            
            ### Question:
            {sample['instruction']}

        """
    
    sample['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    sample['labels'] = tokenizer(sample["response"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return sample

tokenized_train_dataset = train_dataset.map(
    tokenize_function, 
    fn_kwargs={"tokenizer": tokenizer_flan_t5_base},
    # batched=True
)
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['instruction', 'response'])

tokenized_eval_dataset = eval_dataset.map(
    tokenize_function, 
    fn_kwargs={"tokenizer": tokenizer_flan_t5_base},
    # batched=True
)
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(['instruction', 'response'])

tokenized_test_dataset = test_dataset.map(
    tokenize_function, 
    fn_kwargs={"tokenizer": tokenizer_flan_t5_base},
    # batched=True
)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['instruction', 'response'])

print('The output dataset is ready for fine-tuning.')

Map:   0%|          | 0/494 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

Map:   0%|          | 0/69 [00:00<?, ? examples/s]

The output dataset is ready for fine-tuning.


In [17]:

# model_flan_t5_base.config.pretraining_tp = 1

# # Validate that the model is using flash attention, by comparing doc strings
# if use_flash_attention:
#     from utils.llama_patch import forward
#     assert model_llama_2_7b.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"


# # tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)
# tokenizer_flan_t5_base.pad_token = tokenizer_flan_t5_base.eos_token
# tokenizer_flan_t5_base.padding_side = "right"

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

Setup the PEFT/LoRA model for Fine-Tuning

In [18]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



In [22]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, TaskType

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.05,
        r=32,
        bias="none",
        task_type="SEQ_2_SEQ_LM", # Flan T5
)

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

# prepare model for training i.e Add LoRA adapter layers/parameters to the original LLM to be trained.
model_flan_t5_base.enable_input_require_grads()
model_flan_t5_base_peft = prepare_model_for_kbit_training(model_flan_t5_base)
model_flan_t5_base_peft = get_peft_model(model_flan_t5_base_peft, 
                                         peft_config)

print(print_number_of_trainable_model_parameters(model_flan_t5_base_peft))


trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


### Train PEFT Adapter
Define training arguments and create Trainer instance.

Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use.

In [31]:
from transformers import TrainingArguments

peft_training_args = TrainingArguments(
    output_dir="models/flan-t5-fine-tuned",
    num_train_epochs=10,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False # disable tqdm since with packing values are in correct
)



In [32]:
%%time

from trl import SFTTrainer

max_seq_length = tokenizer_flan_t5_base.model_max_length #2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model_flan_t5_base_peft,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer_flan_t5_base,
    packing=True,
    formatting_func=format_instruction,
    args=peft_training_args,
)

trainer.train()

Step,Training Loss
10,0.183100
20,0.150500
30,0.133000
40,0.106400
50,0.096200
60,0.090400
70,0.078900
80,0.078900
90,0.072000
100,0.070300


CPU times: user 4min 44s, sys: 1min 50s, total: 6min 35s
Wall time: 6min 30s


TrainOutput(global_step=450, training_loss=0.05652664343516032, metrics={'train_runtime': 390.3966, 'train_samples_per_second': 12.654, 'train_steps_per_second': 1.588, 'total_flos': 2497308068413440.0, 'train_loss': 0.05652664343516032, 'epoch': 9.73})

In [33]:
# save model
trainer.save_model()

In [28]:
peft_training_args = TrainingArguments(
    output_dir="models/flan-t5-fine-tuned",
    # auto_find_batch_size=True,
    per_device_train_batch_size=6,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=model_flan_t5_base_peft,
    tokenizer=tokenizer_flan_t5_base,
    args=peft_training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

peft_trainer.train()

IndexError: list index out of range

In [ ]:
peft_model_path="models/flan-t5-fine-tuned"

peft_trainer.model.save_pretrained(peft_model_path)
# tokenizer.save_pretrained(peft_model_path)

In [25]:
helper.free_gpu_memory()
helper.memory_stats()

GPU Memory Used: 0.94 GB
GPU Memory Cached: 1.02 GB


In [26]:
helper.check_gpu_capacity()
# del model_llama_2_7b
# del model_llama_2_7b_kbit
helper.free_gpu_memory()
print('------')
helper.memory_stats()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 21165 MB
  Used Memory: 1350 MB
------
GPU Memory Used: 0.94 GB
GPU Memory Cached: 1.02 GB


In [27]:
%%time

# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()


Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

## Test Model and run Inference

In [63]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()

import torch
from peft import AutoPeftModelForCausalLM, AutoPeftModelForSeq2SeqLM, PeftModel, PeftConfig
from transformers import AutoTokenizer

peft_training_args.output_dir = "models/flan-t5-fine-tuned/checkpoint-405"

# load base LLM model and tokenizer
# model = AutoPeftModelForSeq2SeqLM.from_pretrained(
#     peft_training_args.output_dir,
#     low_cpu_mem_usage=True,
#     torch_dtype=torch.float16,
#     load_in_4bit=True,
# )
model = PeftModel.from_pretrained(
    model_flan_t5_base,
    peft_training_args.output_dir,
    torch_dtype=torch.float16,
    is_trainable=False
)
tokenizer = AutoTokenizer.from_pretrained(peft_training_args.output_dir)

In [65]:
nn = 42

instruction = test_dataset[nn]['instruction']
print(instruction)
response = test_dataset[nn]['response']

prompt = f"""### Instruction:
Answer this question.

### Question:
{instruction}

### Answer:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, 
                         do_sample=True, top_p=0.9, temperature=0.2)

# print(f"Prompt:\n{sample_ds['response']}\n")
generated_instruction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
# generated_instruction = generated_instruction.split('\n')[0].strip()
generated_instruction

What are the prerequisites for recipe development?


'cipe that is a recipe that is a recipe that is a recipe that is a recipe that is a recipe that is a recipe that is a recipe that is a recipe that is a recipe that is a recipe that is a recipe that is a recipe that is '

In [66]:
test_dataset[nn]['response']

'Instruction Recipe Development Liquid milks\n\nPurpose\n\nThe objective of this instruction is to ensure that the right steps and requirements are taken into consideration for Recipe Development of Liquid Milks from initial formulation to factory implementation.\n\nScope\n\nThis instruction applies to all recipe developments of EU liquid milks by Product Technologists within PD3 at Nutricia Research.\n\nReferences\n\nAll references in their currently valid version.\n\nPR.06 Recipe Development\n\nIN.06.02 Instruction Bestmix Formulation (Powder)\n\nIN.06.03 Instruction Bestmix Formulation (Liquid & Liquid Cereals)\n\nIN.06.04 Instruction Robustness Check\n\nIN.06.17 Instruction PDS numbering liquid milks\n\nIN.06.19 Premix development Instruction\n\nIN.06.27 Instruction Automated selection of existing premixes or base powders in Bestmix\n\nWD.06.01.01 Mineral Overview\n\nWD.06.01.03 Vitamin Losses overview Table\n\nWD.06.03.01 Template Calculation Premixes for Liquid milks (to be made)

Let’s load the dataset again with a random sample to try to generate an instruction.

In [33]:
# from datasets import load_dataset
# from random import randrange


# # Load dataset from the hub and get a sample
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
# sample = dataset[randrange(len(dataset))]

# prompt = f"""### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# {sample_ds['response']}

# ### Response:
# """

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
4.1.	UV-Light
Together with oxygen, UV-light can produce a singlet oxygen which can form directly a hydroperoxide from a unsaturated lipid (see Figure 3). Usually during the processing of powders, prevention of UV-light intrusion is being controlled. Furthermore, it depends on the package used, but as well how the consumer stores the product. 
4.2.	Oxygen
Reaction with oxygen can take at various sites in the lipid oxidation chain reaction (see Figure 3). Exclusion of oxygen means mainly flushing inert gases typically nitrogen (or argon) to displace oxygen, packaging in vacuum or in modified atmosphere packaging (MAP). Oxygen scavengers can be employed to completely remove residual oxygen from the system. 
4.3.	Heat
Higher temperatures speed up the lipid oxidation reaction rates. Although the product should be prevented from heat, during production heat is needed to: 
1.	be able to handle ingredients (e.g. melting of fat, best done under nitrogen)
2.	get a product microbiologically stable
3.	spray dry the product
Clearly heat is needed during production but where possible it should be minimized to prevent lipid oxidation. 

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.2)

# print(f"Prompt:\n{sample_ds['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
# print(f"Ground truth:\n{sample_ds['instruction']}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Generated instruction:
What are the main factors that cause lipid oxidation in food?
        


### Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 50 instructions and responses to save time). 

In [34]:
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [35]:
import pandas as pd


instruction_test = dataset['instruction'][50:100]
response_test = dataset['response'][50:100]

generated_instruction_list = []
ground_truth_list = []

for instruction, response in zip(instruction_test, response_test):
    prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{response}

### Response:
"""

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    # with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.2)
    
    # print(f"Prompt:\n{sample_ds['response']}\n")
    generated_instruction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    generated_instruction = generated_instruction.split('\n')[0].strip()

    generated_instruction_list.append(generated_instruction)
    ground_truth_list.append(instruction)

zipped_summaries = list(zip(generated_instruction_list, ground_truth_list))
 
df = pd.DataFrame(zipped_summaries, columns = ['generated_instruction_list', 'ground_truth_list'])
df

,generated_instruction_list,ground_truth_list
0,What are the 10 commandments of life?,Think of some family rules to promote a health...
1,Who is Karlon Stark,"In the series A Song of Ice and Fire, who is t..."
2,What is enriched air?,What is enriched air and why would divers dive...
3,What is Casablanca?,What film won the 1943 Oscar as best film
4,Is cold water heavier than hot water?,"which weighs more, cold or hot water?"
5,What is the top speed of a Kia Stinger?,"Given this paragraph, what is the top speed of..."
6,Is it a good idea to have both a cat and a bir...,Write a short paragraph about why you should n...
7,What is the publishing rate of individual stri...,What is your favorite strip from the comic Cal...
8,What are the main aspects that make F1 cars fast?,What makes a formula one car so fast?
9,What is the Key lime pie?,Without quoting directly from the text give me...


In [37]:
for index, row in df[40:50].iterrows():
    print('GI: ', row['generated_instruction_list'])

    print('GT: ',row['ground_truth_list'], '\n')

GI:  What is the difference between Viola toeria and Samphor?
GT:  Identify which instrument is string or percussion: Samphor, Viola toeria 

GI:  What are the things found inside a house?
GT:  If we were playing a game where we had to identify things that can be found inside a house, which of these would we call out: car, chair, table, park, cloud, microwave. 

GI:  What are the different types of beer?
GT:  Classify the following as either dark-colored beers or light colored beers: porter, pilsner, stout, amber, lager 

GI:  
GT:  Give me a bulleted list of the 5 highest mountains in the world and their respective heights in meters 

GI:  • When was Daniel Leavitt born?
GT:  Extract all of the dates mentioned in this paragraph and list them using bullets in the format {Date} - {Description} 

GI:  What products does Apple sell?
GT:  Which products apple sell? 

GI:  What is the moral of the story?
GT:  Write a short story about a person who discovers a hidden room in their house. The

In [38]:
!pip install evaluate==0.4.0 rouge_score==0.1.2 --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Compute ROUGE score for this subset of the data. 

In [39]:
import evaluate

rouge = evaluate.load('rouge')

model_results = rouge.compute(
    predictions=generated_instruction_list,
    references=ground_truth_list[0:len(generated_instruction_list)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODEL:')
model_results_df = pd.DataFrame(model_results.items(), columns=['Metric', 'Value'])
model_results_df

MODEL:


,Metric,Value
0,rouge1,0.467613
1,rouge2,0.297052
2,rougeL,0.436774
3,rougeLsum,0.434922
